In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/Shareddrives/'CSCE 638 NLP'

/content/gdrive/Shareddrives/CSCE 638 NLP


In [3]:
!ls

'9_10_2022_Who Will R.pdf'
 all_data
 all_question_data.zip
 annotated_question_intimacy_data
 explained.html
 Extra_dataset
 Gender_detection.ipynb
 gender_predictor.pkl
 gender_predictor_v2.pkl
'NLP Project_initimacy_score.ipynb'
'NLP Project_kunal.ipynb'
 NLP_project_visualization.ipynb
 outputs
 outputs-20221130T175107Z-001.zip
 outputs_question_initmacy_semeval
'Project Abstract.gdoc'
'Project - Social intimacy'
'Sujith_Deberta-NLP Project_template.ipynb'
 test_output
 test_prediction_final.txt
'Topic detection.ipynb'
 train_all.txt
 train.csv
 train.gsheet
 train.txt


### Install Dependencies

In [4]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 40.6 MB/s 
     |████████████████████████████████| 182 kB 67.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 


In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### Define args here such as model to use and path to dataset

In [6]:
class Args:

  model_name = 'xlnet-base-cased'
  pre_trained_model_name_or_path = 'xlnet-base-cased'
  train_path = "./all_data/train_train.txt"
  val_path = "./all_data/train_validate.txt"
  test_path = "./all_data/train_test.txt"
  model_saving_path = "./outputs"
  test_saving_path = "./test_output/test.txt"

  # Model options:
  # model_name = 'roberta-base'
  # pre_trained_model_name_or_path = 'roberta-base'
  # model_name = 'pedropei/question-intimacy'
  # pre_trained_model_name_or_path = 'pedropei/question-intimacy'
  # model_name = 'xlm-roberta-base'
  # pre_trained_model_name_or_path = 'xlm-roberta-base'

  # dataset options
  # train_path=data/annotated_question_intimacy_data/final_train.txt \
  # val_path=data/annotated_question_intimacy_data/final_val.txt \
  # test_path=data/annotated_question_intimacy_data/final_test.txt \
  # model_saving_path = "./outputs"
  # test_saving_path = "./test_output/test.txt"

def arguments():
    parser = ArgumentParser()
    parser.set_defaults(show_path=False, show_similarity=False)

    parser.add_argument('--model_name')
    parser.add_argument('--pre_trained_model_name_or_path')
    parser.add_argument('--train_path', default='train.txt')
    parser.add_argument('--val_path', default='val.txt')
    parser.add_argument('--test_path', default='test.txt')
    parser.add_argument('--model_saving_path', default=None)
    parser.add_argument('--test_saving_path', default=None)

    return parser.parse_args()


### Set Hyperparameters

In [7]:
class Config(object):
	def __init__(self, model_name=None):
		self.model_name = model_name
		self.max_epochs = 50
		self.lr = 0.00002
		self.batch_size = 128
		self.cuda = True
		self.max_len = 50
		self.warmup_ratio = 0.06
		self.weight_decay=0.0
		self.gradient_accumulation_steps = 1
		self.adam_epsilon = 1e-08

### Dataset preprocessing

In [8]:
'''
Convert csv to txt
'''
def convert():
  import csv
  import random
  csv_file = 'train.csv' # 'Extra_dataset/reddit_post_questions.csv'
  out_file = 'train.txt' # 'Extra_dataset/reddit_post_questions.txt'
  with open(out_file, "w") as f_out:
      with open(csv_file, "r") as f_in:
        fr = csv.reader(f_in)
        next(fr)
        for row in fr:
          txt, score = row[0], float(row[1])
          # txt, score = row[1], float(row[2])
          # score = 1 + 4*((score+1)/2) # convert from [-1, 1] to [1, 5]
          if row[2]=='English':
            f_out.write(txt+" "+ str(score) + '\n')
      f_out.close()

'''
Merging different datasets
'''
def combine():
  input_files = ['train.txt','reddit.txt','movie.txt','question.txt']
  output_file = 'train_all.txt'

  with open(output_file, 'w') as outfile:
      for names in input_files:
          with open(names) as infile:
              outfile.write(infile.read())
          outfile.write("\n")

'''
Shuffling the data
'''
def shuffle():
  input_file = 'train_all.txt'
  output_file = 'train_shuffled.txt'

  lines = open(input_file).readlines()
  random.shuffle(lines)
  open(output_file, 'w').writelines(lines)

'''
Splitting the data into train, test and validate datasets
80-10-10 split
'''
def split():
  train = 0
  validate = 0

  input_file = 'train_shuffled.txt'
  train_output_file = 'all_train.txt'
  validate_output_file = 'all_validate.txt'
  test_output_file = 'all_test.txt'

  with open(input_file, 'r') as file:
      lines = file.readlines()
      train = int(0.8*len(lines))
      validate = int(0.9*len(lines))

  with open(train_output_file, 'w') as file:
      for line in lines[:train]:
          file.write(line)

  with open(validate_output_file, 'w') as file:
      for line in lines[train:validate]:
          file.write(line)

  with open(test_output_file, 'w') as file:
      for line in lines[validate:]:
          file.write(line)


In [9]:
def get_data(path):
    print('open:',path)
    text = []
    y = []
    with open(path, 'r') as r:
        lines = r.readlines()
        for line in lines:
            line = line.strip('\n').split(' ')
            try:
              # y.append(float(line[-1]))
              y.append(((float(line[-1])/2)-1.5))
            except Exception as e:
              # print(line)
              print("Error parsing line: ", line)
              continue
            text.append(' '.join(line[:-1]))
    return text,y

### Helper functions to run epochs

In [10]:
# train.py

import math
from scipy import stats
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torch import nn, Tensor
from torch.autograd import Variable
import torch
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaForSequenceClassification, XLNetModel, XLNetTokenizer, XLNetForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
from argparse import ArgumentParser


torch.manual_seed(0)


def padding(text, pad, max_len = 50):

    return text if len(text) >= max_len else (text + [pad] * (max_len-len(text)))

def encode_batch(text, berts, max_len = 50):
    tokenizer = berts[0]
    t1 = []
    for line in text:
        t1.append(padding(tokenizer.encode(line,add_special_tokens = True, max_length = max_len,truncation=True),tokenizer.pad_token_id,max_len))
    return t1


def data_iterator(train_x, train_y, batch_size = 64):
    n_batches = math.ceil(len(train_x) / batch_size)
    for idx in range(n_batches):
        x = train_x[idx *batch_size:(idx+1) * batch_size]
        y = train_y[idx *batch_size:(idx+1) * batch_size]
        yield x, y

def run_epoch(model, train_data, val_data, tokenizer,config, optimizer):
    train_x, train_y = train_data[0], train_data[1]
    val_x, val_y = val_data[0], val_data[1]
    iterator = data_iterator(train_x, train_y, config.batch_size)
    train_losses = []
    val_accuracies = []
    losses = []

    for i, (x,y) in tqdm(enumerate(iterator),total=int(len(train_x)/config.batch_size)):
        #print('iteration', i) 
        model.zero_grad()

        ids = encode_batch(x, (tokenizer,model), max_len = config.max_len)


        if config.cuda:
            input_ids = Tensor(ids).cuda().long()
            labels = torch.cuda.FloatTensor(y)
        else:
            input_ids = Tensor(ids).long()
            labels = torch.FloatTensor(y)

        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

        loss.backward()
        #print('train_loss',loss)
        losses.append(loss.data.cpu().numpy())
        optimizer.step()

        if (i + 1) % 1 == 0:
            #print("Iter: {}".format(i))
            avg_train_loss = np.mean(losses)
            train_losses.append(avg_train_loss)
            #print("\tAverage training loss: {:.5f}".format(avg_train_loss))
            losses = []

            # Evalute Accuracy on validation set
            model.eval()
            all_preds = []
            val_iterator = data_iterator(val_x, val_y, config.batch_size)
            for x, y in val_iterator:
                ids = encode_batch(x, (tokenizer,model), max_len = config.max_len)
                #x = Variable(Tensor(x))

                with torch.no_grad():

                    if config.cuda:
                        input_ids = Tensor(ids).cuda().long()
                        labels = torch.cuda.FloatTensor(y)
                    else:
                        input_ids = Tensor(ids).long()
                        labels = torch.FloatTensor(y)
                    outputs = model(input_ids, labels=labels)
                    loss, y_pred = outputs[:2]

                predicted = y_pred.cpu().data

                all_preds.extend(predicted.numpy())

            
            all_res = np.array(all_preds).flatten()            
            score = (np.square(val_y - all_res)).mean()
            val_accuracies.append(score)
            model.train()

    return train_losses, val_accuracies

def get_test_result(model, test_x, test_y, config,tokenizer, save_path, ext_test = False, pure_inference=False):
    cuda = config.cuda
    all_raw = []
    all_preds = []
    all_y = []
    all_x = []
    test_iterator = data_iterator(test_x, test_y, batch_size=256)
    model.eval()
    i = 0
    for x, y in test_iterator:
        print(str(i * 256) + '/' + str(len(test_x)))
        i += 1
        #print(x[:5])
        ids = encode_batch(x, (tokenizer, model), max_len = config.max_len)
        # x = Variable(Tensor(x))

        with torch.no_grad():
            if cuda:
                input_ids = Tensor(ids).cuda().long()
                #labels = torch.cuda.FloatTensor(y)
            else:
                input_ids = Tensor(ids).long()
                #labels = torch.FloatTensor(y)
            outputs = model(input_ids)
            y_pred = outputs[0]

        predicted = y_pred.cpu().data
        #predicted = torch.max(y_pred.cpu().data, 1)[1]
        all_preds.extend(predicted.numpy())
        #all_raw.extend(y_pred.cpu().data.numpy())
        all_y.extend(y)
        all_x.extend(x)


    #all_res = [1 if i[0] > 0 else -1 for i in all_preds]
    all_res = np.array(all_preds).flatten()
    #all_raw = np.array(all_raw)

    if save_path:
        with open(save_path, 'w') as w:
            if pure_inference:
                 for i in range(len(all_y)):                
                    if i < 2:
                        print(all_x[i], all_res[i])                    
                    w.writelines(all_x[i] + '\t' + str(all_res[i]) + '\n')
            else:
                for i in range(len(all_y)):                
                    if i < 2:
                        print(all_x[i], all_res[i], test_y[i])                    
                    w.writelines(all_x[i] + '\t' + str(all_y[i]) + '\t' + str(all_res[i]) + '\n')
    
    if not pure_inference:
        print('mse:', (np.square(all_y - all_res)).mean())
        print('pearson r:', stats.pearsonr(all_res, all_y)[0])

    if ext_test:
        print('book pearson r:', stats.pearsonr(all_res[:50], all_y[:50])[0])
        print('twitter pearson r:', stats.pearsonr(all_res[50:100], all_y[50:100])[0])
        print('movie pearson r:', stats.pearsonr(all_res[100:150], all_y[100:150])[0])

    return all_res, all_y




Helper function to determine Pearson R and MSE

In [11]:
def get_metrics(model, test_x, test_y, config, tokenizer, test = False, save_path='test_prediction_final.txt'):
    cuda = config.cuda
    all_preds = []
    test_iterator = data_iterator(test_x, test_y, batch_size=64)
    all_y = []
    all_x = []
    model.eval()
    for x, y in test_iterator:
        ids = encode_batch(x, (tokenizer,model), max_len = config.max_len)
        with torch.no_grad():
            if cuda:
                input_ids = Tensor(ids).cuda().long()                
                labels = torch.cuda.FloatTensor(y)
            else:
                input_ids = Tensor(ids).long()
                labels = torch.FloatTensor(y)
            outputs = model(input_ids, labels=labels)
            loss, y_pred = outputs[:2]

        predicted = y_pred.cpu().data
        all_preds.extend(predicted.numpy())
        all_y.extend(y)
        all_x.extend(x)

    all_res = np.array(all_preds).flatten()
    if test and save_path:
        with open(save_path, 'w') as w:
            for i in range(len(all_y)):
                if i < 2:
                    print(all_x[i], all_res[i], test_y[i])
                w.writelines(all_x[i] + '\t' + str(all_y[i]) + '\t' + str(all_res[i]) + '\n')

    #print('pearson r:', stats.pearsonr(all_res, all_y)[0])
    score = 0
    return loss,stats.pearsonr(all_res, all_y)[0]

Helper functions for visualizations


In [12]:
import matplotlib
import matplotlib.colors as colors
from IPython.core.display import display, HTML

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

def colorize(inputs, color_arrays):
    cmap = truncate_colormap(matplotlib.cm.YlOrRd, maxval=0.9)
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for words, color_array in zip(inputs, color_arrays):
        for i, (word, color) in enumerate(zip(words.split(), color_array)):
            # print(word, color)
            color = matplotlib.colors.rgb2hex(cmap(color)[:3])
            colored_string += template.format(color, '&nbsp' + word + '&nbsp')
            if not (i+1)%25:
                colored_string += '<br>'
        # colored_string += '<p>next tweet</p>'
        colored_string += '<br>'

    with open('explained.html', 'w') as f:
       f.write(colored_string)
    display(filename='explain.html')

### Main function

In [13]:
args = Args()
config = Config(args.model_name)
# Change tokenizer import and model import when trying different model
tokenizer = XLNetTokenizer.from_pretrained(args.pre_trained_model_name_or_path, num_labels=1, output_attentions = False,output_hidden_states = False)
# tokenizer = RobertaTokenizer.from_pretrained(args.pre_trained_model_name_or_path, num_labels=1, output_attentions = False,output_hidden_states = False)
# tokenizer = AutoTokenizer.from_pretrained(args.pre_trained_model_name_or_path, num_labels=1, output_attentions = False,output_hidden_states = False)
# tokenizer = XLMRobertaTokenizer.from_pretrained(args.pre_trained_model_name_or_path, num_labels=1, output_attentions = False,output_hidden_states = False)
# Create Model with specified optimizer and loss function
##############################################################

# model = AutoModelForSequenceClassification.from_pretrained(args.pre_trained_model_name_or_path,num_labels=1,output_attentions = False,output_hidden_states = False)
# model = XLMRobertaForSequenceClassification.from_pretrained(args.pre_trained_model_name_or_path,num_labels=1,output_attentions = False,output_hidden_states = False)
model = XLNetForSequenceClassification.from_pretrained(args.pre_trained_model_name_or_path,num_labels=1,output_attentions = False,output_hidden_states = False)
if config.cuda:
    model.cuda()
      
          
train_text,  train_labels = get_data(args.train_path)
val_text,  val_labels = get_data(args.val_path)
test_text,  test_labels = get_data(args.test_path)
# print("text: ", test_text[:5])
# print("labels:", test_labels[:5])
# print(len(test_text), "---", len(test_labels))

train_x = train_text
train_y = np.array(train_labels)
val_x = val_text
val_y = np.array(val_labels)
model.train()
optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=1e-6)
##############################################################

train_data = [train_x, train_y]
val_data = [val_x, val_y]

# Get Accuracy of final model
test_x = test_text
test_y = np.array(test_labels)
best_val = 100.0
best_test = 100.0
best_r = 100

for i in range(config.max_epochs):
    print ("Epoch: {}".format(i))

    train_losses,val_accuracies = run_epoch(model, train_data, val_data, tokenizer,config, optimizer)
    test_acc,test_r = get_metrics(model, test_x, test_y, config, tokenizer, test = True, save_path=args.test_saving_path)
    #print('Final Test Accuracy: {:.4f}'.format(test_acc))

    print("\tAverage training loss: {:.5f}".format(np.mean(train_losses)))
    print("\tAverage Val MSE: {:.4f}".format(np.mean(val_accuracies)))
    if np.mean(val_accuracies) < best_val:
        best_val = np.mean(val_accuracies)
        best_test = test_acc
        best_r = test_r
        if i >= 1 and args.model_saving_path:
            model.save_pretrained(args.model_saving_path)
            tokenizer.save_pretrained(args.model_saving_path)


print('model saved at', args.model_saving_path)
print('best_val_loss:', best_val)
print('best_test_loss:',best_test)
print('best_test_pearsonr:',best_r)

test_acc = get_metrics(model, test_x, test_y, config, tokenizer, test = True)
if args.model_saving_path:
    model.save_pretrained(args.model_saving_path)
    tokenizer.save_pretrained(args.model_saving_path)




Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

open: ./all_data/train_train.txt
open: ./all_data/train_validate.txt
Error parsing line:  ['']
open: ./all_data/train_test.txt
Epoch: 0


...

@user I MEANT TO PUT NO -0.8844712 -1.0
@user Studying for my exam to get my teaching license! -0.59373754 -0.6
	Average training loss: 0.02313
	Average Val MSE: 0.1421
Epoch: 48


10it [00:19,  1.92s/it]


@user I MEANT TO PUT NO -0.86040634 -1.0
@user Studying for my exam to get my teaching license! -0.592407 -0.6
	Average training loss: 0.02181
	Average Val MSE: 0.1451
Epoch: 49


10it [00:19,  1.91s/it]


@user I MEANT TO PUT NO -0.8271264 -1.0
@user Studying for my exam to get my teaching license! -0.47427586 -0.6
	Average training loss: 0.02115
	Average Val MSE: 0.1477
model saved at ./outputs
best_val_loss: 0.13466132375722012
best_test_loss: tensor(0.0795, device='cuda:0')
best_test_pearsonr: 0.6771716153790823
@user I MEANT TO PUT NO -0.8271264 -1.0
@user Studying for my exam to get my teaching license! -0.47427586 -0.6


### Get Results for val and test data set

In [14]:
print('val:')
test_result, test_score = get_test_result(model, val_text, val_labels, config, tokenizer,save_path=None, ext_test = False)

print('test:')
test_result, test_score = get_test_result(model, test_text, test_labels, config, tokenizer,save_path=None, ext_test = False)

val:
0/158
mse: 0.14451733594467536
pearson r: 0.629146366411491
test:
0/159
mse: 0.11370948169498203
pearson r: 0.6687940415656368


### Print attention score visualization

In [15]:
from IPython.core.display import display, HTML

print("explaining ...")
def visualize_attention(input_text, layer=-1):

    inputs = tokenizer(input_text, return_tensors='pt')
    # print(inputs)
    output = model(inputs['input_ids'].to(device), output_attentions=True)
    attention = output.attentions[layer]
    # print(attention.shape)
    weights = attention[0, 0, :len(input_text.split()), 0].cpu().detach().numpy()
    assert len(input_text.split()) == len(weights)

    return weights
train_text,  train_labels = get_data(args.train_path)
color_arrays = []
texts = []
for txt, label in zip(train_text, train_labels):
    if label > 0:
      weights = visualize_attention(txt)
      texts.append(txt)
      color_arrays.append(weights)
# print(texts)
colorize(texts, color_arrays)

html = open('explained.html', "r")
html_text = "".join(html)
print(html_text)

# display()
display({'text/html': html_text}, raw=True)

explaining ...
open: ./all_data/train_train.txt
<span class="barcode"; style="color: black; background-color: #fffec9">&nbsp’m&nbsp</span><span class="barcode"; style="color: black; background-color: #ffffcb">&nbspnot&nbsp</span><span class="barcode"; style="color: black; background-color: #fffec9">&nbspashamed&nbsp</span><span class="barcode"; style="color: black; background-color: #fffeca">&nbspof&nbsp</span><span class="barcode"; style="color: black; background-color: #ffffcb">&nbspwhat&nbsp</span><span class="barcode"; style="color: black; background-color: #ffffcb">&nbspI&nbsp</span><span class="barcode"; style="color: black; background-color: #fffeca">&nbspam&nbsp</span><span class="barcode"; style="color: black; background-color: #fffec9">&nbspand&nbsp</span><span class="barcode"; style="color: black; background-color: #fffeca">&nbspthat&nbsp</span><span class="barcode"; style="color: black; background-color: #ffffcc">&nbspI&nbsp</span><span class="barcode"; style="color: black;

’m   not   ashamed   of   what   I   am   and   that   I   have   curves,   and   that   I’m   thick.   I   loke   my   body   💕   http   @user   『✿』   You   should   message   them   about   that!   I'm   sure   most   people   would   love   to   RP   with   you.   Nothing   I   like   more   than   knowing   my   friends   are   happy   😍🥺❤️   The   less   likely   I   am   to   be   with   you   the   more   I   want   you   i   give   up   I   care.   I   always   care.   Masturbation   is   key.   Not   Dylan   getting   my   sloppy   2nd   just   bc   we   fucked   him   does   not   mean   we   are   the   same   hoe   @user   @NASA   I   identify   as   c*nt..it's   more   of   a   gender   slur   but   I   must   be   called   by   it.   I'm   a   sadist.   I   love   when   my   bff   come   over.   Best   conversations   ever,   my   dawg   fasho.   🤞🏾💕👭   i   would   love   to   be   dead   rn   @user   @user   Alexa   is   my   main.   #AlexaIlacad   |   @user   99   @user   was   that   YOU   dancing   in   ur   recent   story???!!!   😯   Another   weekend   and   Love   never   find   you,   sigh😔   Ladies   and   Gentlemen   What   part   in   your   opposite   sex   can   make   you   react   like   this   Me:   That   🍑   😂   http   sex   ....   i   wanna   have   some😩   wife   riding   strangers   cock   http   @ArianaGrande   take   care   of   yourself.   i   love   u   sm   Thick   AF   Latina   MILF   with   great   tits   and   DSLs   #MILF   #BigTits   #BigAss   #NSFW   http   I   want   a   man   that’s   just   orgasmic   and   just   understand   me   and   even   if   he   don’t   still   respects   me   and   loves   it   all   like   MMM.   the   professor   i’m   in   love   w   canceled   our   exam   next   thursday   🥰   fuck   daddy   issues   who   got   mommy   AND   daddy   issues   All   I   want   is   a   girl   with   a   Phat   Ass   who   can   ride   dick   like   @user   http   i   jus   wish   my   ass   was   fat   snd   my   stomach   was   flat   🙄   @user   sis   didn't   you   get   your   nipples   pierced???   ur   the   bravest   woman   alive   already.   whats   a   lil   ear   needle   🤷‍♀️   @user   🤣🤣🤣🤣thank   you   TakaTina   my   baby   kicked...kķkkķk   was   kinda   worried   the   whole   morning   Nothing   compares   with   being   with   someone   who   actually   cares   for   you   How   can   you   simply   be   friends   with   someone   when   every   time   you   look   at   them   all   you   think   about   is   how   much   more   you   really   want   them?   @user   Hello   baby❤️,   I   would   love   to   buy   your   feet   pictures   and   videos,   DM   if   interested.   Bitches   will   get   so   lonely   they’ll   turn   gay   🤦🏽‍♂️   You   can’t   even   be   nice   to   a   nigga   without   his   dick   getting   hard   ugh   ..   I   really   hate   y’all   If   you   would   let   me   give   you   pinky   promise   kisses   If   I   could   I   would   unfuck   98%   of   yu   whores🎯   @user   I   see   you   have   been   tearing   it   up!   Very   proud   of   you!   (Your   middle   school   coach)   4   Months   into   a   ceasefire   with   @user   be   like…   #IAMKONG   #SOON   👀   🏀   🚀   http   @user   Can   I   have   you   lol   nobody   goes   through   more   shit   in   life   than   a   person   with   a   good   heart   Do   we   really   need   to   be   in   a   relationship😊   Lust   for   porn   http   @user   Fuck   your   feet   are   SO   SEXY   omg   Your   need   to   be   used   as   a   Fuckdoll   is   absolute   @user   http   I   just   can’t   have   sex   with   just   anyone   lol   ppl   are   wild   good   sucking   huge   cock   http   @user   This   was   such   a   nice   compliment   I   really   appreciate   this!   Thank   you!!   need   a   kiss   😔   @user   Get   sucking   cunt   my   mistress   wants   her   money   @user   @user   Your   beauty   likes   the   ancient   Egyptian   phar